In [ ]:
import json
import os
from collections import defaultdict
from IPython.display import HTML
import numpy as np
import matplotlib
from matplotlib import animation
import matplotlib.pyplot as plt

from analysis import (
    parse_results, get_token_groups, plot_token_groups, animate, average_length_by_weight,
    tokens, counter_tokens, total_of, sum_up, mk_cumulative_interpolator
)
%matplotlib inline

In [ ]:
def read(path):
    tout, Cout, info = parse_results(path)
    tokg, ctrg, spg = get_token_groups(Cout)
    init = []
    for sk in spg['polymeric']:
        if Cout[sk][0] > 0:
            init.append((sk, Cout[sk][0]))
    (sk0, ic0), = init
    return locals()

In [ ]:
token = 'W3'
cases = __import__('cases_%s' % token)
res = lambda **kw: ('%s-res/%s_{scal}_{mf}_{atmo}.txt' % (token, token)).format(**kw)
mfs = range(len(cases.mass_fracs))
mfs = [0, 3]  # for now...  # number 2 contained an error

In [ ]:
dur, varied = json.load(open('varied-%s.json' % token))
dr = varied.pop('doserate')
assert not varied
cx, cy, ip1d = mk_cumulative_interpolator(dur, dr)

In [ ]:
all_air, all_nit = {}, {}
for scal, in cases.crn_combos:
    for mf, atmo in cases.int_combos:
        if atmo == 0:
            r = all_air
        elif atmo == 1:
            r = all_nit
        else:
            raise ValueError("Unexpected atmosphere index")
        r[scal, mf] = read(res(scal=scal, mf=mf, atmo=atmo))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
for i, k in enumerate(all_air[0, 0]['spg']['ord_rad']):
    ax.loglog(all_air[0, 0]['tout'], all_air[0, 0]['Cout'][k], ls=['-',':','-.'][i % 3], label=k)
ax.set_ylim([1e-15, 100])
ax.legend()

In [ ]:
#plot_token_groups(tout, Cout, tokg)

In [ ]:
#anim1 = animate(tout, Cout, tokg)
#HTML(anim1.to_html5_video())

In [ ]:
def plot_conc(series_air, series_nit, sks='H2O2 O2 N2O HO2'.split()):
    fig, all_axes = plt.subplots(len(series_air), 1 + len(sks), figsize=(16, 3*len(series_air)), sharey='col')
    assert len(series_air) == len(series_nit)
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
            axes[0].axhline(int(cont['sk0'][1:].split('a')[0]), ls='--', lw=0.5, alpha=0.5, c='k')
            axes[0].plot(ip1d(cont['tout'])/1e3, average_length_by_weight(cont['Cout'], cont['tokg']), label=lbl)
            for i in range(len(sks)):
                axes[i+1].plot(ip1d(cont['tout'])/1e3, cont['Cout'][sks[i]], label=lbl, alpha=0.7)
                axes[i+1].set_title(sks[i])
                axes[i+1].set_yscale('log')
        for ax in axes:
            ax.set_xlabel('Dose / kGy')
        axes[0].legend()
        axes[1].legend()
        _ = axes[0].set_title(r'Average length by weight')

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
if token == 'W2':
    for scal in range(5):
        plot_conc([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif token == 'W3':
    plot_conc(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])

In [ ]:
# This cell generates figures for Mats' conference presentation
def _plot_all(series_air, series_nit):
    assert len(series_air) == len(series_nit)
    for mfi, (air, nit) in enumerate(zip(series_air, series_nit)):
        ppm = cases.mass_fracs[mfi] * 1e6
        fig1, ax1 = plt.subplots(1, 1)
        for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
            n0 = int(cont['sk0'][1:].split('a')[0])  # intial lenght
            ax1.axhline(n0, ls='--', lw=0.5, alpha=0.5, c='k')
            ax1.plot(ip1d(cont['tout'])/1e3, average_length_by_weight(cont['Cout'], cont['tokg']), label=lbl)
        ax1.set_ylabel('Chain length / number of segments')
        ax1.set_xlabel('Dose / kGy')
        ax1.legend()
        fig1.savefig('poly_Poland_size_%dppm.png' % ppm, dpi=300)
        
        for specie in ['H2O2', 'e-(aq)', 'N2O']:
            fig2, ax2 = plt.subplots(1, 1)
            for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
                ax2.plot(ip1d(cont['tout'])/1e3, cont['Cout'][specie], label=lbl, alpha=0.7)
            ax2.set_ylabel(r'[$\mathrm{%s}$] / M' % {'H2O2': 'H_2O_2', 'e-(aq)': 'e^-(aq)', 'N2O': 'N_2O'}[specie])
            ax2.set_xlabel('Dose / kGy')
            ax2.ticklabel_format(axis='y', style='sci', scilimits=(-4, 4))
            ax2.set_yscale('log')
            ax2.set_ylim([1e-8, 1e-4])
            ax2.legend()
            fig2.tight_layout()
            fig2.savefig('poly_Poland_%s_%dppm.png' % (specie, ppm), dpi=300)
        
        fig3, ax3 = plt.subplots(1, 1)
        for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
            ax3.plot(ip1d(cont['tout']), sum_up(cont['Cout'], cont['ctrg']['unsaturated'])/cont['ic0'], label=lbl)
        ax3.set_ylabel('Number of double bonds per polymer')
        ax3.set_xlabel('Dose / kGy')
        ax3.legend()
        fig3.savefig('poly_Poland_unsaturated_%dppm.png' % ppm, dpi=300)
_plot_all(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])

In [ ]:
def plot_tokens(series_air, series_nit):
    assert len(series_air) == len(series_nit)
    fig, all_axes = plt.subplots(len(series_air), len(tokens[1:]), figsize=(16, 2*len(series_air)), sharey='col')
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for tk, ax in zip(tokens[1:], axes):
            for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
                ax.plot(ip1d(cont['tout']), total_of(cont['Cout'], cont['tokg'][tk])/cont['ic0'], label=lbl)
            ax.legend()
            ax.set_title(tk)
            ax.set_xlabel('Dose / Gy')
            ax.set_yscale('log')

In [ ]:
if token == 'W2':
    for scal in range(5):
        plot_tokens([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif token == 'W3':
    plot_tokens(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])

In [ ]:
def plot_counters(series_air, series_nit):
    assert len(series_air) == len(series_nit)
    fig, all_axes = plt.subplots(len(series_air), len(counter_tokens), figsize=(16, 2*len(series_air)), sharey='col')
    for axes, air, nit in zip(all_axes, series_air, series_nit):
        for ct, ax in zip(counter_tokens, axes):
            for lbl, cont in list(zip(['air', 'nitrous oxide'], [air, nit])):
                ax.plot(ip1d(cont['tout']), sum_up(cont['Cout'], cont['ctrg'][ct])/cont['ic0'], label=lbl)
            ax.set_xlabel('Dose / Gy')
            ax.legend()
            ax.set_title(ct)

In [ ]:
if token == 'W2':
    for scal in range(5):
        plot_counters([all_air[scal, 0], all_air[scal, 1]], [all_nit[scal, 0], all_nit[scal, 1]])
elif token == 'W3':
    plot_counters(*[[cont[0, mf] for mf in mfs] for cont in [all_air, all_nit]])